In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sys
from tqdm import tnrange
from pathlib import Path

In [ ]:
sys.path.append('../model')

In [ ]:
from efficiency import ValueSet, efficiency
from models import SimpleCNN2Layer as Model
from training import select_gpu
from collectdata import collect_data
import torch
from loss import Loss

In [ ]:
folder = Path('Aug_16_120000_2layer')
name = 'Aug_16_120000_2layer'

#folder = Path('../model/output3')
#name = '20180815_120000_3layer'

In [ ]:
#with np.load(folder / f'{name}_stats.npz') as stats:
#    cost = stats['cost']
#    eff_val = stats['eff_val']
#    valid = stats['val']
#    
#eff_rate = np.empty(len(eff_val))
#for n,s in enumerate(eff_val):
#    eff_rate[n] = ValueSet(*s).eff_rate

In [ ]:
device = select_gpu(0)
model = Model()
model.eval()
model.to(device)
loss = Loss()

In [ ]:
batch_size = 32

trainfile = Path('/share/lazy/schreihf/PvFinder/Aug_14_80K.npz')
valfile = Path('/share/lazy/schreihf/PvFinder/Oct03_20K_val.npz')

train_loader = collect_data(trainfile, batch_size=batch_size, device=device, shuffle=True)
val_loader = collect_data(valfile, batch_size=batch_size, device=device, shuffle=False)

In [ ]:
results = np.empty([6, 80])
for i in tnrange(80):
    model.load_state_dict(torch.load(folder / f'{name}_{i}.pyt'))
    with torch.no_grad():
        gpu_outputs = model(validation.dataset.tensors[0])
        outputs = gpu_outputs.cpu().numpy()
        labels = validation.dataset.tensors[1].cpu().numpy()
        val_loss = loss(gpu_outputs, validation.dataset.tensors[1]).data.item()
        
        gpu_train_outputs = model(training.dataset.tensors[0])
        train_outputs = gpu_train_outputs.cpu().numpy()
        train_labels = training.dataset.tensors[1].cpu().numpy()
        train_loss = loss(gpu_train_outputs, training.dataset.tensors[1]).data.item()
    
    train = ValueSet(0,0,0,0)
    val = ValueSet(0,0,0,0)

    for label, output in zip(labels, outputs):
        val += efficiency(label, output, 5., 1e-2, .2, 3)

    for label, output in zip(train_labels, train_outputs):
        train += efficiency(label, output, 5., 1e-2, .2, 3)
     
    results[0, i] = train.eff_rate
    results[1, i] = val.eff_rate
    results[2, i] = train.fp_rate
    results[3, i] = val.fp_rate
    results[4, i] = train_loss
    results[5, i] = val_loss

In [ ]:
120000 / 32 

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

ax.set_title(name)
ax.plot(results[4] / (10000 / 32), label='Training')
ax.plot(results[5] / (10000 / 32), label='Validation')
ax.set_ylim(-3,3)
ax.set_yticks(np.linspace(0,3,4))
ax.set_yticks(np.linspace(0,3,13), minor=True)
ax.set_ylabel(format('Cost', '>40'))
ax.set_xlabel('Epoch')
ax.grid(which='both', axis='y')
ax.legend()

tax = ax.twinx()
tax.plot(results[0])
tax.plot(results[1])
tax.set_ylim(0, 1.6)
tax.set_yticks(np.linspace(0,.8,9))
tax.set_ylabel(format('Efficiency', '<46'));
tax.grid(axis='y')

In [ ]:
fig, ax = plt.subplots()

ax.plot(results[2], label='Training')
ax.plot(results[3], label='Validation')

ax.set_title(name)
ax.set_ylabel('FP / event')
ax.set_xlabel('Epoch')
ax.legend();